In [ ]:
import os
import shutil
import alpacapella

In [ ]:
ANNOTATION_PATH = "../dataset"
name = """\
looperman-a-2596114-0018761-acapella-hook-brazilian-trap-fake-friends
"""[:-1]
ACAPELLA_PATH = f"../dataset/looperman_sub/{name}.mp3"

annotation = alpacapella.pipeline(ANNOTATION_PATH, smoothing_size=2.2, voting_window=0.04)
alpacapella.play(ACAPELLA_PATH, annotation)


In [ ]:
DATASET_PATH = "../dataset/dataset"
b = input("beat in bar: ")
alpacapella.write_dataset(ACAPELLA_PATH, DATASET_PATH, annotation, beats_in_bar=int(b), cutoff=2)

# remove the rest
DESTINATION = "../dataset/used"
l = len([x for x in os.listdir(DESTINATION) if not x.startswith('.')]) + 1
os.makedirs(os.path.join(DESTINATION, str(l)))
for i, file in enumerate(os.listdir(ANNOTATION_PATH)):
    if not file.endswith('.txt'):
        continue
    source = os.path.join(ANNOTATION_PATH, file)
    sink = os.path.join(DESTINATION, f"{l}", file)
    shutil.move(source, sink)
    os.rename(sink, os.path.join(DESTINATION, f"{l}", f"{name}{i}.txt"))
os.remove(ACAPELLA_PATH)

In [ ]:
# import pandas as pd
# def convert_txt_to_beats(txt_path: str, beats_path: str):
#     df = pd.read_csv(txt_path)
    
#     with open(beats_path, 'w') as f:
#         for _, row in df.iterrows():
#             time = row['TIME']
#             label = row['LABEL']
#             beat_in_bar = int(str(label).split('.')[1])
#             f.write(f"{time:.9f} {beat_in_bar}\n")

# def convert_folder_txt_to_beats(folder_path: str):
#     txt_files = [f for f in os.listdir(folder_path) if f.startswith('annotation') and f.endswith('.txt')]
    
#     for txt_file in txt_files:
#         txt_path = os.path.join(folder_path, txt_file)
#         beats_file = txt_file.replace('.txt', '.beats')
#         beats_path = os.path.join(folder_path, beats_file)
#         convert_txt_to_beats(txt_path, beats_path)

# convert_folder_txt_to_beats(DATASET_PATH)